In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import tqdm
from scipy.stats import levene

# import anova and tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
def anova_function(features_df: pd.DataFrame, Metdata_column: str) -> pd.DataFrame:
    """
    This function will take in a dataframe and a metadata column and return the results of an anova and tukeyhsd test for each feature.


    Parameters
    ----------
    features_df : pd.DataFrame
        The dataframe containing the features with only one metadata column
    Metdata_column : str
        The name of the metadata column to be used for the anova test

    Returns
    -------
    pd.DataFrame
        A dataframe containing the results of the anova and tukeyhsd test for each feature
    """

    # anova and tukeyhsd for each feature
    # create a pandas data frame to store the results
    anova_results = pd.DataFrame()

    # loop through each feature
    for feature in tqdm.tqdm(features_df.columns[:-1]):
        # create a model
        model = ols(f"{feature} ~ C({Metdata_column})", data=features_df).fit()
        # create an anova table
        anova_table = sm.stats.anova_lm(model, typ=2)
        # create a tukeyhsd table
        tukeyhsd = pairwise_tukeyhsd(features_df[feature], features_df[Metdata_column])
        # get the f-statistic based p-value
        anova_p_value = anova_table["PR(>F)"][0]
        tmp = pd.DataFrame(
            tukeyhsd._results_table.data, columns=tukeyhsd._results_table.data[0]
        ).drop(0)
        tmp.reset_index(inplace=True, drop=True)
        # drop the first row
        tmp["feature"] = feature
        tmp["anova_p_value"] = anova_p_value
        tmp = pd.DataFrame(tmp)

        anova_results = pd.concat([anova_results, tmp], axis=0).reset_index(drop=True)
    return anova_results

In [3]:
file_path = pathlib.Path("../../data/5.converted_data/mean_aggregated_data.parquet")
df = pd.read_parquet(file_path)
df.head()

,Metadata_genotype,Metadata_replicate,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,AreaShape_CentralMoment_1_2,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,high,1,0.596483,0.596483,0.270876,-0.049237,0.390868,2.100915,-0.463937,-0.042982,...,-0.104105,-0.119036,-0.545856,-0.525027,-0.543305,-0.544589,-0.557563,-0.562101,-0.558976,-0.560872
1,high,10,1.238954,1.238954,0.659813,1.722364,-0.874297,0.179195,-1.644760,0.825270,...,0.646977,0.635356,0.630682,0.642091,0.593754,0.635274,0.557863,0.548473,0.560985,0.547769
2,high,11,2.131207,2.131207,0.630866,1.426469,0.291591,1.801207,-0.270662,-0.164307,...,0.686771,0.678230,0.552382,0.570823,0.526266,0.548475,0.498825,0.486269,0.500083,0.490213
3,high,12,1.201238,1.201238,0.024943,1.279119,-0.464086,-1.652853,-0.982309,0.107680,...,0.309505,0.308933,-0.054779,-0.059876,-0.098400,-0.067930,-0.103782,-0.109367,-0.101086,-0.106772
4,high,13,1.194888,1.194888,-0.554368,-0.034466,0.050882,-1.344980,-0.483673,-0.794434,...,0.056706,0.047741,-0.325171,-0.295674,-0.321039,-0.322436,-0.348409,-0.356255,-0.350286,-0.352168


In [4]:
# split the features and the metadata
metadata = df.columns.str.contains("Metadata")
# filter the metadata
metadata_df = df.loc[:, metadata]
# filter the features
features_df = df.loc[:, ~metadata]

## Anova for genotype only

In [5]:
anova_input_df = features_df.copy()
anova_input_df["Metadata_genotype"] = metadata_df["Metadata_genotype"]
anova_output_df = anova_function(anova_input_df, "Metadata_genotype")
print(anova_output_df.shape)
anova_output_df.head()

  0%|                                                                                                                          | 0/244 [00:00<?, ?it/s]

  0%|▍                                                                                                                 | 1/244 [00:00<00:30,  7.95it/s]

  1%|▉                                                                                                                 | 2/244 [00:00<00:29,  8.08it/s]

  1%|█▍                                                                                                                | 3/244 [00:00<00:29,  8.22it/s]

  2%|█▊                                                                                                                | 4/244 [00:00<00:29,  8.17it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 243/244 [00:30<00:00,  7.85it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:30<00:00,  7.88it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:30<00:00,  8.02it/s]

(732, 9)


,group1,group2,meandiff,p-adj,lower,upper,reject,feature,anova_p_value
0,high,unsel,-1.4484,0.0,-1.9694,-0.9273,True,AreaShape_Area,3.794260e-11
1,high,wt,-2.0158,0.0,-2.5369,-1.4948,True,AreaShape_Area,3.794260e-11
2,unsel,wt,-0.5674,0.0302,-1.0885,-0.0464,True,AreaShape_Area,3.794260e-11
3,high,unsel,-1.4484,0.0,-1.9694,-0.9273,True,AreaShape_CentralMoment_0_0,3.794260e-11
4,high,wt,-2.0158,0.0,-2.5369,-1.4948,True,AreaShape_CentralMoment_0_0,3.794260e-11


In [6]:
# save the results
output_file = pathlib.Path(
    "../../data/6.analysis_results/mean_aggregated_anova_results.parquet"
)
output_file.parent.mkdir(exist_ok=True, parents=True)
anova_output_df.to_parquet(output_file)

## Levene's test for homogeneity of variance

In [7]:
# split the df into three genotypes
high_df = df[df["Metadata_genotype"] == "high"]
unsel_df = df[df["Metadata_genotype"] == "unsel"]
wt_df = df[df["Metadata_genotype"] == "wt"]
levene_test_results = {"feature": [], "levene_statistic": [], "levene_p_value": []}
for feature in tqdm.tqdm(features_df.columns):
    # calculate the levene test for each feature
    levene_results = levene(wt_df[feature], unsel_df[feature], high_df[feature])
    levene_test_results["feature"].append(feature)
    levene_test_results["levene_statistic"].append(levene_results.statistic)
    levene_test_results["levene_p_value"].append(levene_results.pvalue)

levene_test_results_df = pd.DataFrame(levene_test_results)
levene_test_results_df

  0%|                                                                                                                          | 0/244 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:00<00:00, 3985.52it/s]

,feature,levene_statistic,levene_p_value
0,AreaShape_Area,2.370155,0.106798
1,AreaShape_CentralMoment_0_0,2.370155,0.106798
2,AreaShape_CentralMoment_0_1,2.489193,0.096071
3,AreaShape_CentralMoment_0_2,2.858310,0.069441
4,AreaShape_CentralMoment_0_3,0.938341,0.399931
...,...,...,...
239,Texture_SumVariance_OP_3_03_256,6.397836,0.003954
240,Texture_Variance_OP_3_00_256,6.746330,0.003047
241,Texture_Variance_OP_3_01_256,7.062972,0.002411
242,Texture_Variance_OP_3_02_256,6.696927,0.003161


## Calculate the levenes test statistic for the equality of variances

In [8]:
# split the df into three genotypes
high_df = df[df["Metadata_genotype"] == "high"]
unsel_df = df[df["Metadata_genotype"] == "unsel"]
wt_df = df[df["Metadata_genotype"] == "wt"]
group_dict = {
    "high_vs_unsel": [high_df, unsel_df],
    "high_vs_wt": [high_df, wt_df],
    "unsel_vs_wt": [wt_df, unsel_df],
    "all": [high_df, unsel_df, wt_df],
}


levene_test_results = {
    "feature": [],
    "levene_statistic": [],
    "levene_p_value": [],
    "group": [],
}
for group in tqdm.tqdm(group_dict.keys()):
    for feature in features_df.columns:
        # calculate the levene test for each feature
        if not group == "all":
            levene_results = levene(
                group_dict[group][0][feature], group_dict[group][1][feature]
            )
            levene_test_results["feature"].append(feature)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)
        else:
            levene_results = levene(
                group_dict[group][0][feature],
                group_dict[group][1][feature],
                group_dict[group][2][feature],
            )
            levene_test_results["feature"].append(feature)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)

levene_test_results_df = pd.DataFrame(levene_test_results)

# sort the levene test results levene_test_results_df
# change the levene p-value to a float
levene_test_results_df["levene_p_value"] = levene_test_results_df[
    "levene_p_value"
].astype(float)
levene_test_results_df = levene_test_results_df.sort_values(
    "levene_p_value", ascending=False
)
levene_test_results_df

  0%|                                                                                                                            | 0/4 [00:00<?, ?it/s]

/home/lippincm/miniconda3/envs/op_cell_processing_env/lib/python3.8/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom


 75%|███████████████████████████████████████████████████████████████████████████████████████                             | 3/4 [00:00<00:00, 22.32it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 21.78it/s]

,feature,levene_statistic,levene_p_value,group
128,RadialDistribution_MeanFrac_OP_3of4,0.000006,9.981062e-01,high_vs_unsel
909,RadialDistribution_ZernikePhase_OP_6_6,0.001928,9.980735e-01,all
0,AreaShape_Area,0.000050,9.944103e-01,high_vs_unsel
1,AreaShape_CentralMoment_0_0,0.000050,9.944103e-01,high_vs_unsel
56,AreaShape_SpatialMoment_0_0,0.000050,9.944103e-01,high_vs_unsel
...,...,...,...,...
168,RadialDistribution_ZernikePhase_OP_4_0,65.524194,1.417644e-08,high_vs_unsel
656,RadialDistribution_ZernikePhase_OP_4_0,65.524194,1.417644e-08,unsel_vs_wt
268,AreaShape_HuMoment_4,75.236081,3.756747e-09,high_vs_wt
900,RadialDistribution_ZernikePhase_OP_4_0,65.524194,3.385289e-13,all


In [9]:
# save the levene test results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
levene_test_results_path = pathlib.Path(
    out_dir / "mean_aggregated_levene_test_results.csv"
)
levene_test_results_df.to_csv(levene_test_results_path)

### Calculate the levene test statistic for the aggregated data across feature types and genotypes

In [10]:
data_path = pathlib.Path(
    "../../data/5.converted_data/mean_aggregated_data.parquet"
).resolve(strict=True)
# Read the data
data = pd.read_parquet(data_path)

# Drop all metadata except for the genotype data
features_df = data.drop(columns=data.filter(like="Metadata").columns)
features_df["Metadata_genotype"] = data["Metadata_genotype"]


# turn the features into a long format
features_long_df = features_df.melt(
    id_vars="Metadata_genotype", var_name="feature", value_name="value"
)
features_long_df.head()
# Separate the feature into different parts
features_long_df[
    ["feature_group", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
] = features_long_df["feature"].str.split("_", expand=True)

# Replace the Metadata_genotype with the actual genotype name
features_long_df["Metadata_genotype"] = features_long_df["Metadata_genotype"].replace(
    {"high": "High-Severity", "unsel": "Mid-Severity", "wt": "Wild Type"}
)
features_long_df.head()

,Metadata_genotype,feature,value,feature_group,measurement,bone,parameter1,parameter2,parameter3
0,High-Severity,AreaShape_Area,0.596483,AreaShape,Area,None,None,None,None
1,High-Severity,AreaShape_Area,1.238954,AreaShape,Area,None,None,None,None
2,High-Severity,AreaShape_Area,2.131207,AreaShape,Area,None,None,None,None
3,High-Severity,AreaShape_Area,1.201238,AreaShape,Area,None,None,None,None
4,High-Severity,AreaShape_Area,1.194888,AreaShape,Area,None,None,None,None


In [11]:
# break each genotype and featuretype into a separate dataframe
high_df = features_long_df[features_long_df["Metadata_genotype"] == "High-Severity"]
unsel_df = features_long_df[features_long_df["Metadata_genotype"] == "Mid-Severity"]
wt_df = features_long_df[features_long_df["Metadata_genotype"] == "Wild Type"]

# each feature group
high_df_AreaShape = high_df[high_df["feature_group"] == "AreaShape"]
high_df_Intensity = high_df[high_df["feature_group"] == "Intensity"]
high_df_Neighbors = high_df[high_df["feature_group"] == "Neighbors"]
high_df_radial = high_df[high_df["feature_group"] == "RadialDistribution"]
high_df_Granularity = high_df[high_df["feature_group"] == "Granularity"]

unsel_df_AreaShape = unsel_df[unsel_df["feature_group"] == "AreaShape"]
unsel_df_Intensity = unsel_df[unsel_df["feature_group"] == "Intensity"]
unsel_df_Neighbors = unsel_df[unsel_df["feature_group"] == "Neighbors"]
unsel_df_radial = unsel_df[unsel_df["feature_group"] == "RadialDistribution"]
unsel_df_Granularity = unsel_df[unsel_df["feature_group"] == "Granularity"]

wt_df_AreaShape = wt_df[wt_df["feature_group"] == "AreaShape"]
wt_df_Intensity = wt_df[wt_df["feature_group"] == "Intensity"]
wt_df_Neighbors = wt_df[wt_df["feature_group"] == "Neighbors"]
wt_df_radial = wt_df[wt_df["feature_group"] == "RadialDistribution"]
wt_df_Granularity = wt_df[wt_df["feature_group"] == "Granularity"]

# levene test for each feature group
levene_test_results = {
    "feature_group": [],
    "levene_statistic": [],
    "levene_p_value": [],
    "group": [],
}

group_dict = {
    "AreaShape": {
        "high_area_v_unsel_area": [high_df_AreaShape, unsel_df_AreaShape],
        "high_area_v_wt_area": [high_df_AreaShape, wt_df_AreaShape],
        "unsel_area_v_wt_area": [wt_df_AreaShape, unsel_df_AreaShape],
    },
    "Intensity": {
        "high_intensity_v_unsel_intensity": [high_df_Intensity, unsel_df_Intensity],
        "high_intensity_v_wt_intensity": [high_df_Intensity, wt_df_Intensity],
        "unsel_intensity_v_wt_intensity": [wt_df_Intensity, unsel_df_Intensity],
    },
    "Neighbors": {
        "high_neighbors_v_unsel_neighbors": [high_df_Neighbors, unsel_df_Neighbors],
        "high_neighbors_v_wt_neighbors": [high_df_Neighbors, wt_df_Neighbors],
        "unsel_neighbors_v_wt_neighbors": [wt_df_Neighbors, unsel_df_Neighbors],
    },
    "RadialDistribution": {
        "high_radial_v_unsel_radial": [high_df_radial, unsel_df_radial],
        "high_radial_v_wt_radial": [high_df_radial, wt_df_radial],
        "unsel_radial_v_wt_radial": [wt_df_radial, unsel_df_radial],
    },
    "Granularity": {
        "high_granularity_v_unsel_granularity": [
            high_df_Granularity,
            unsel_df_Granularity,
        ],
        "high_granularity_v_wt_granularity": [high_df_Granularity, wt_df_Granularity],
        "unsel_granularity_v_wt_granularity": [wt_df_Granularity, unsel_df_Granularity],
    },
}

for feature_group in tqdm.tqdm(group_dict.keys()):
    for group in group_dict[feature_group].keys():
        if not group == "all":
            levene_results = levene(
                group_dict[feature_group][group][0]["value"],
                group_dict[feature_group][group][1]["value"],
            )
            # calculate the variance for each feature group

            levene_test_results["feature_group"].append(feature_group)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)
        else:
            pass

levene_test_results_df = pd.DataFrame(levene_test_results)
levene_test_results_df.head()

  0%|                                                                                                                            | 0/5 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1537.95it/s]

,feature_group,levene_statistic,levene_p_value,group
0,AreaShape,188.439067,1.593028e-41,high_area_v_unsel_area
1,AreaShape,558.839674,1.330891e-112,high_area_v_wt_area
2,AreaShape,42.952365,6.676022e-11,unsel_area_v_wt_area
3,Intensity,37.096223,2.551443e-09,high_intensity_v_unsel_intensity
4,Intensity,0.816115,3.668389e-01,high_intensity_v_wt_intensity


In [12]:
# save the levene test results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
levene_test_results_path = pathlib.Path(
    out_dir / "mean_aggregated_levene_test_results_feature_types.csv"
)
levene_test_results_df.to_csv(levene_test_results_path, index=False)

In [13]:
# Drop all metadata except for the genotype data
features_df = data.drop(columns=data.filter(like="Metadata").columns)
features_df["Metadata_genotype"] = data["Metadata_genotype"]

# turn the features into a long format
features_long_df = features_df.melt(
    id_vars="Metadata_genotype", var_name="feature", value_name="value"
)
features_long_df

,Metadata_genotype,feature,value
0,high,AreaShape_Area,0.596483
1,high,AreaShape_Area,1.238954
2,high,AreaShape_Area,2.131207
3,high,AreaShape_Area,1.201238
4,high,AreaShape_Area,1.194888
...,...,...,...
10243,wt,Texture_Variance_OP_3_03_256,1.324503
10244,wt,Texture_Variance_OP_3_03_256,1.062536
10245,wt,Texture_Variance_OP_3_03_256,1.247334
10246,wt,Texture_Variance_OP_3_03_256,1.590171


In [14]:
# get the variance for each feature group
var_df = features_long_df.groupby(["Metadata_genotype", "feature"]).var().reset_index()
var_df.head()
# change the value column name to variance
var_df.rename(columns={"value": "variance"}, inplace=True)

In [15]:
var_df[
    ["feature_group", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
] = var_df["feature"].str.split("_", expand=True)

# Replace the Metadata_genotype with the actual genotype name
var_df["Metadata_genotype"] = var_df["Metadata_genotype"].replace(
    {"high": "High-Severity", "unsel": "Mid-Severity", "wt": "Wild Type"}
)
var_df
var_df = var_df.drop(
    columns=["feature", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
)
var_df
# save the variance results
var_path = pathlib.Path(out_dir / "mean_aggregated_variance_results_feature_types.csv")
var_df.to_csv(var_path, index=False)

In [16]:
# get the mean and stdev for each feature group's variance
var_df = (
    var_df.groupby(["Metadata_genotype", "feature_group"])
    .agg(["mean", "std", "max", "min", "count"])
    .reset_index()
)
# ungroup the columns
var_df.columns = ["_".join(col).strip() for col in var_df.columns.values]
# rename the Metadata_genotype_ column and the feature_group_ column
var_df.rename(
    columns={
        "Metadata_genotype_": "Metadata_genotype",
        "feature_group_": "feature_group",
    },
    inplace=True,
)
var_df

,Metadata_genotype,feature_group,variance_mean,variance_std,variance_max,variance_min,variance_count
0,High-Severity,AreaShape,0.424204,0.350670,2.057226,0.000002,98
1,High-Severity,Granularity,0.118054,0.137097,0.384985,0.010917,6
2,High-Severity,Intensity,0.330695,0.192987,0.832953,0.049040,15
3,High-Severity,Neighbors,0.477229,0.183397,0.667070,0.301038,3
4,High-Severity,RadialDistribution,0.381793,0.258899,1.301865,0.000000,70
5,High-Severity,Texture,0.198174,0.127244,0.425544,0.014608,52
6,Mid-Severity,AreaShape,0.538798,0.535868,3.162029,0.073452,98
7,Mid-Severity,Granularity,0.184305,0.124284,0.419402,0.095894,6
8,Mid-Severity,Intensity,0.266804,0.258139,1.057746,0.055703,15
9,Mid-Severity,Neighbors,0.526392,0.250131,0.815115,0.375350,3


In [17]:
# save the variance results
var_path = pathlib.Path(
    out_dir / "mean_aggregated_variance_results_feature_types_stats.csv"
)
var_df.to_csv(var_path, index=False)